## Asyncio

### Generate all even numbers

In [2]:
def integers(start=1, step=1):
  a = start
  while True:
    yield a
    a += step

In [3]:
def evens():
    return integers(start=2, step=2)

In [5]:
from itertools import islice 
list(islice(evens(), 0, 10))

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

or, reusing integers from the course

In [21]:
def integers():
  a = 1
  while True:
    yield a
    a += 1


def even_integers():
    for a in integers():
        if a % 2 == 0:
            # return a # <-- this fails 
            yield a  

In [22]:
list(islice(even_integers(), 0, 10))

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

### Generate fibonacci numbers

In [23]:
def fibonacci():
    f_n1 = 0 
    f_n2 = 1 
    while True:
        out = f_n1 + f_n2
        yield out
        f_n2 = f_n1
        f_n1 = out

In [24]:
list(islice(fibonacci(), 0, 10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

### using yield to send stuff

In [30]:
def printer():
    i = 1
    while True:
        x = yield
        print(f"{i:03}: {x}")
        i += 1

p = printer()
next(p)   # we need to advance the coroutine to the first yield
p.send("Mercury")
p.send("Venus")
p.send("Earth")

001: Mercury
002: Venus
003: Earth


(futures = tasks (?))

In [31]:
import asyncio

async def counter(name):
  for i in range(5):
    print(f"{name:<10} {i:03}")
    await asyncio.sleep(0.2) # the task here is to sleep

await counter("Venus")

Venus      000
Venus      001
Venus      002
Venus      003
Venus      004


In [32]:
await asyncio.gather(counter("Earth"), counter("Moon"))

Earth      000
Moon       000
Earth      001
Moon       001
Earth      002
Moon       002
Earth      003
Moon       003
Earth      004
Moon       004


[None, None]

In [34]:
!python -m notebooks.async_counter # note the bug in the lesson materials

Earth      000
Moon       000
Earth      001
Moon       001
Earth      002
Moon       002
Earth      003
Moon       003
Earth      004
Moon       004


Timing async code

In [35]:
from dataclasses import dataclass
from typing import Optional
from time import perf_counter
from contextlib import asynccontextmanager


@dataclass
class Elapsed:
    time: Optional[float] = None


@asynccontextmanager
async def timer():
    e = Elapsed()
    t = perf_counter()
    yield e
    e.time = perf_counter() - t

In [36]:
async with timer() as t:
  await asyncio.sleep(0.2)
print(f"that took {t.time} seconds")

that took 0.20066843899985543 seconds


### Compute pi again

In [37]:
import random
import numba


@numba.njit(nogil=True)
def calc_pi(N):
    M = 0
    for i in range(N):
        # Simulate impact coordinates
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # True if impact happens inside the circle
        if x**2 + y**2 < 1.0:
            M += 1
    return 4 * M / N

In [39]:
async with timer() as t:
    await asyncio.to_thread(calc_pi, 10**7)

print(f"that took {t.time} seconds")

that took 0.06433338900023955 seconds
